# GetResultsFromElasticSearch

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from datetime import datetime
import os
import pandas as pd
import csv

In [2]:
ELASTICSEARCH_HOST=os.getenv('ELASTICSEARCH_HOST')
ELASTICSEARCH_USERNAME=os.getenv('ELASTICSEARCH_USERNAME')
ELASTICSEARCH_PASSWORD=os.getenv('ELASTICSEARCH_PASSWORD')

In [3]:
ELASTICSEARCH_PASSWORD

'owuus9neTe'

In [4]:
# OPEN CONNECTION
es = Elasticsearch([ELASTICSEARCH_HOST], http_auth=(ELASTICSEARCH_USERNAME, ELASTICSEARCH_PASSWORD))

In [5]:
def parseElasticSearchDate(timestamp):
    return datetime.strptime(timestamp, '%b %d, %Y @ %H:%M:%S.%f')

## Preliminar

In [6]:
def getMetrics(metric,start, finish):
    search = Search(using=es, index="metricbeat-*") \
        .filter('range' ,  **{'@timestamp': {'gte': start , 'lt': finish, 'time_zone': '+02:00' }}) \
        .filter('term', **{'fields.node_role': 'browseremulator'}) \
        .filter('term', **{'metricset.name': metric})
    total = search.count()
    return search[0:total].execute()

In [7]:
def getJoinedTimestamps(start, finish):
    search = Search(using=es, index="openvidu") \
        .filter('range' ,  **{'timestamp': {'gte': start , 'lt': finish, 'time_zone': '+02:00', 'format': 'strict_date_optional_time' }}) \
        .query(Q('match', event="participantJoined"))
    total = search.count()
    return search[0:total].execute()

In [8]:
def saveResults(results, strategy, participants, filename):
    filepath = "Preliminar/{numP}_Participants_Per_Session/{strategy}/{numP}_{strategy}_{filename}.csv".format(
        strategy=strategy,
        numP=participants,
        filename=filename
    )
    with open(filepath,"w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(results)

In [25]:
df = pd.read_csv('Preliminar/strategies.csv', sep="\t")
for index, row in df.iterrows():
    print(row['strategy'])
    
    start = parseElasticSearchDate(row['start'])
    finish = parseElasticSearchDate(row['finish'])
    
    # CPU
    cpu_results_raw = getMetrics("cpu", start, finish)
    cpu_results = [['@timestamp','system.cpu.total.norm.pct']]
    for hit in cpu_results_raw :
        cpu = hit['system']['cpu']['total']['norm']['pct'] * 100
        timestamp = datetime.strptime(hit['@timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ')
        cpu_results.append([timestamp, cpu])
    saveResults(cpu_results, row['strategy'], row['participants'], "CPU")
        
    # MEM
    mem_results_raw = getMetrics("memory", start, finish)
    mem_results = [['@timestamp','system.memory.used.pct']]
    for hit in mem_results_raw:
        mem = hit['system']['memory']['used']['pct'] * 100
        timestamp = datetime.strptime(hit['@timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ')
        mem_results.append([timestamp, mem])
    saveResults(mem_results, row['strategy'], row['participants'], "Mem")
    
    # JOINED 
    joined_results_raw = getJoinedTimestamps(start, finish)
    joined_results = [['timestamp','event']]
    for hit in joined_results_raw:
        timestamp = datetime.fromtimestamp(hit['timestamp']/1000.0)
        joined_results.append([timestamp, hit['event']])
    saveResults(joined_results, row['strategy'], row['participants'], "Joined")

kms-webrtc
kms-webrtc
kms-webrtc
kms-webrtc
SeleniumWithoutRecording
SeleniumWithoutRecording
SeleniumWithoutRecording
SeleniumWithoutRecording
SeleniumWithRecording
SeleniumWithRecording
SeleniumWithRecording
SeleniumWithRecording
headless browsers
headless browsers
headless browsers
headless browsers


In [23]:
datetime.strptime(hit_cpu['@timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ')

datetime.datetime(2021, 9, 30, 16, 46, 22, 264000)